In [1]:
import numpy as np
import pandas as pd

In [2]:
aisles_df = pd.read_csv('../data/original/aisles.csv')
departments_df = pd.read_csv('../data/original/departments.csv')

order_products_prior_df = pd.read_csv('../data/original/order_products__prior.csv')
order_products_train_df = pd.read_csv('../data/original/order_products__train.csv')

orders_df = pd.read_csv('../data/original/orders.csv')
products_df = pd.read_csv('../data/original/products.csv')

In [3]:
aisles_df.isnull().any()

aisle_id    False
aisle       False
dtype: bool

In [4]:
departments_df.isnull().any()

department_id    False
department       False
dtype: bool

In [5]:
order_products_prior_df.isnull().any()

order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool

In [6]:
order_products_train_df.isnull().any()

order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool

In [7]:
len(order_products_prior_df.product_id.unique())

49677

In [8]:
orders_df.isnull().any()

order_id                  False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
dtype: bool

#### 向上合并days_since_prior_order为0的order
1.在order_df中,删除为0的order条目，重置order_number  
2.在all_order_products_df中，将为0的order内的所有商品的order_id 改成上一条的order_id 如果有重复就不修改直接丢弃
add_to_cart_order 顺序累加，

In [10]:
since_prior_zero_df = orders_df[orders_df['days_since_prior_order'] == 0]
all_order_products_df = pd.concat([order_products_prior_df, order_products_train_df])

In [11]:
len(all_order_products_df.product_id.unique())

49685

In [15]:
# i = 8
# prior_series = orders_df.loc[i - 1]
# zero_series = orders_df.loc[i]

# #获取对应order_id的商品列表
# zero_df = all_order_products_df[all_order_products_df['order_id'] == zero_series.order_id]
# prior_df = all_order_products_df[all_order_products_df['order_id'] == prior_series.order_id]

# # 删除common_products
# common_products = set(prior_df['product_id']) & set(zero_df['product_id'])
# drop_indexes = []
# for p in common_products:
#     index = zero_df[zero_df['product_id'] == p].index
#     drop_indexes.extend(index)
# all_order_products_df.drop(drop_indexes, inplace=True)

# #修改order_id:修改成proir_series的id
# change_indexs = all_order_products_df[all_order_products_df['order_id'] == zero_series.order_id].index
# all_order_products_df.loc[change_indexs, 'order_id'] = prior_series.order_id

# #修改add_to_cart_order,延续proir_id赋值
# start_add_to_cart_order = np.max(prior_df['add_to_cart_order']) + 1
# for j in change_indexs:
#     all_order_products_df.loc[j, 'add_to_cart_order'] = start_add_to_cart_order
#     start_add_to_cart_order = start_add_to_cart_order + 1

In [9]:
# c = 0
# #将days_since_prior_order为0的order中的商品合并到上一个order中
# for i in since_prior_zero_df.index:
#     if c % 500 == 0:
#         print("stage 1 : %d"%c)
#     c = c + 1
#     prior_series = orders_df.loc[i - 1]
#     zero_series = orders_df.loc[i]

#     #获取对应order_id的商品列表
#     zero_df = all_order_products_df[all_order_products_df['order_id'] == zero_series.order_id]
#     prior_df = all_order_products_df[all_order_products_df['order_id'] == prior_series.order_id]

#     # 删除common_products
#     common_products = set(prior_df['product_id']) & set(zero_df['product_id'])
#     drop_indexes = []
#     for p in common_products:
#         index = zero_df[zero_df['product_id'] == p].index
#         drop_indexes.extend(index)
#     all_order_products_df.drop(drop_indexes, inplace=True)

#     #修改order_id:修改成proir_series的id
#     change_indexs = all_order_products_df[all_order_products_df['order_id'] == zero_series.order_id].index
#     all_order_products_df.loc[change_indexs, 'order_id'] = prior_series.order_id

#     #修改add_to_cart_order,延续proir_id赋值
#     start_add_to_cart_order = np.max(prior_df['add_to_cart_order']) + 1
#     for j in change_indexs:
#         all_order_products_df.loc[j, 'add_to_cart_order'] = start_add_to_cart_order
#         start_add_to_cart_order = start_add_to_cart_order + 1

# print("stage 2 start")
# #删除days_since_prior_order为0的order
# orders_df.drop(since_prior_zero_df.index, inplace=True)
# #重置order_number
# orders_df.sort_values(['user_id', 'order_number'], inplace=True)
# max_user_id = np.max(orders_df['user_id'])
# for user in range(1, max_user_id + 1):
#     sub_orders_df = orders_df[orders_df['user_id'] == user]
#     k = 1
#     for j in sub_orders_df.index:
#         orders_df.loc[j, 'order_number'] = k
#         k = k + 1
# print("stage 2 end")

# #重新拆分order_products_prior_df 和 order_products_train_df
# c = 0
# prior_order_products = pd.DataFrame(columns=all_order_products_df.columns)
# train_order_products = pd.DataFrame(columns=all_order_products_df.columns)
# for i in orders_df.index:
#     if c % 1000 == 0:
#         print("stage 3 : %d"%c)
#     c = c + 1
#     sub_order_product_df = all_order_products_df[all_order_products_df['order_id'] == orders_df.loc[i]['order_id']]
#     if orders_df.loc[i]['eval_set'] == 'prior':
#         prior_order_products = pd.concat([prior_order_products, sub_order_product_df])
#     elif orders_df.loc[i]['eval_set'] == 'train':
#         train_order_products = pd.concat([train_order_products, sub_order_product_df])

### 将proir中每个人的所有订单中所有的reordered商品作为判别项

In [12]:
order_order_prouduct_df = pd.merge(order_products_prior_df, orders_df, left_on='order_id', right_on='order_id')

In [21]:
order_order_prouduct_user_groupby = order_order_prouduct_df.groupby('user_id')

In [31]:
user_reordered_df = order_order_prouduct_df.groupby('user_id').apply(lambda t : t[t.reordered == 1].sort_values("order_number"))

In [35]:
user_reordered_df[user_reordered_df['user_id'] == 1]

order_id  product_id  add_to_cart_order  reordered  user_id  \
user_id                                                                         
1       22742748   2398795       26088                  5          1        1   
        22742746   2398795       12427                  3          1        1   
        22742744   2398795         196                  1          1        1   
        4488095     473747         196                  1          1        1   
        4488096     473747       12427                  2          1        1   
        4488097     473747       10258                  3          1        1   
        21376077   2254736       25133                  4          1        1   
        21376078   2254736       26405                  5          1        1   
        21376074   2254736         196                  1          1        1   
        21376075   2254736       12427                  2          1        1   
        21376076   2254736       10258                  3          1        1   
        4089398     431534         196                  1          1        1   
        4089399     431534       12427                  2          1        1   
        4089400     431534       10258                  3          1        1   
        4089401     431534       25133                  4          1        1   
        4089405     431534       13176                  8          1        1   
        31927070   3367565         196                  1          1        1   
        31927071   3367565       12427                  2          1        1   
        31927072   3367565       10258                  3          1        1   
        31927073   3367565       25133                  4          1        1   
        5212927     550135         196                  1          1        1   
        5212928     550135       10258                  2          1        1   
        5212931     550135       13032                  5          1        1   
        5212930     550135       25133                  4          1        1   
        5212929     550135       12427                  3          1        1   
        29474808   3108588       25133                  4          1        1   
        29474807   3108588       10258                  3          1        1   
        29474806   3108588         196                  2          1        1   
        29474805   3108588       12427                  1          1        1   
        21760443   2295261       49235                  1          1        1   
        21760445   2295261       25133                  3          1        1   
        21760448   2295261       12427                  6          1        1   
        21760447   2295261       10258                  5          1        1   
        21760446   2295261         196                  4          1        1   
        21760444   2295261       46149                  2          1        1   
        24181273   2550362       13032                  8          1        1   
        24181274   2550362       12427                  9          1        1   
        24181270   2550362       25133                  5          1        1   
        24181267   2550362       46149                  2          1        1   
        24181266   2550362         196                  1          1        1   
        24181271   2550362       10258                  6          1        1   

                 eval_set  order_number  order_dow  order_hour_of_day  \
user_id                                                                 
1       22742748    prior             2          3                  7   
        22742746    prior             2          3                  7   
        22742744    prior             2          3                  7   
        4488095     prior             3          3                 12   
        4488096     prior             3          3                 12   
        4488097     pri

In [23]:
order_order_prouduct_user_product_count_df = order_order_prouduct_user_groupby['product_id'].value_counts()

In [28]:
order_order_prouduct_user_product_count_df

user_id  product_id
1        196           10
         12427         10
         10258          9
         25133          8
         13032          3
         46149          3
         13176          2
         26088          2
         26405          2
         49235          2
         10326          1
         14084          1
         17122          1
         30450          1
         35951          1
         38928          1
         39657          1
         41787          1
2        32792          9
         47209          8
         24852          7
         1559           6
         18523          6
         19156          6
         12000          5
         16589          5
         33754          5
         2002           4
         19051          4
         22124          4
                       ..
206209   12376          1
         14727          1
         14870          1
         14901          1
         16066          1
         17500          1
         19734    

In [47]:
idx = orders_df[orders_df['eval_set'] == 'prior'].groupby('user_id')['order_number'].idxmax()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
9,2550362,1,prior,10,4,8,30.0
24,839880,2,prior,14,3,10,13.0
37,1402502,3,prior,12,1,15,15.0
43,2557754,4,prior,5,5,13,0.0
48,157374,5,prior,4,1,18,19.0
52,998866,6,prior,3,2,18,12.0
73,2452257,7,prior,20,3,9,7.0
77,2570360,8,prior,3,5,0,30.0
81,1830137,9,prior,3,5,12,30.0
87,1353310,10,prior,5,5,20,23.0


In [42]:
orders_df[orders_df['eval_set'] == 'prior'].groupby('user_id')['order_number'].transform(max)

0          10
1          10
2          10
3          10
4          10
5          10
6          10
7          10
8          10
9          10
11         14
12         14
13         14
14         14
15         14
16         14
17         14
18         14
19         14
20         14
21         14
22         14
23         14
24         14
26         12
27         12
28         12
29         12
30         12
31         12
           ..
3421051    49
3421052    49
3421053    49
3421054    49
3421055    49
3421056    49
3421057    49
3421058    49
3421059    49
3421060    49
3421061    49
3421062    49
3421063    49
3421064    49
3421065    49
3421066    49
3421067    49
3421069    13
3421070    13
3421071    13
3421072    13
3421073    13
3421074    13
3421075    13
3421076    13
3421077    13
3421078    13
3421079    13
3421080    13
3421081    13
Name: order_number, Length: 3214874, dtype: int64

In [35]:
last_order_number_df.index

RangeIndex(start=0, stop=206209, step=1)

In [31]:
last_orders = []

for i in last_order_number_df.index:
    series = last_order_number_df.loc[i]
    last_orders.append(orders_df[(orders_df['user_id'] == series['user_id']) & (orders_df['order_number'] == series['order_number'])])

In [29]:
last_order_id_df = orders_df[orders_df['order_number'].isin(last_order_number_df[''])]